In [1]:
#Step1: Determine temperature trends over the past 5 years

import geohash
from datetime import datetime

def parseLine(line):
    variables = line.split("\t")
    try:
        lat = float(variables[1])
        lon = float(variables[2])
        tem = float(variables[10])
        
        ts = int(variables[0][0: 10])
        # if you encounter a "year is out of range" error the timestamp
        # may be in milliseconds, try `ts /= 1000` in that case
        year = datetime.utcfromtimestamp(ts).strftime('%Y')
        
        gh = geohash.encode(lat, lon)
        return (gh[0: 2] + '\t' + year, tem)
    except:
        return ('', 0)
    
#text_file = spark.read.load('hdfs://orion11:21001/3hr_sample/sampled_2015/*', format='csv', sep='\t', inferSchema=True, header=True)
text_fileCC = sc.textFile("hdfs://orion11:21001/3hr_sample/*")

# (GeoHash, wind_energy_factor, cloud_cover)
parsed_dataCC = text_fileCC \
    .map(lambda line: parseLine(line))

av_data = parsed_dataCC.map(lambda x: (x[0], x[1])) \
    .mapValues(lambda x: (x, 1)) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1]+y[1])) \
    .mapValues(lambda v: (v[0] / v[1])) \
    .sortBy(lambda x: x[0], True)



In [2]:
def parseLine(line):
    try:
        key = line[0]
        value = line[1]
        
        print(key)
        print(value)
        
        variables = key.split("\t")
        year = variables[1]
        geo = variables[0]
        
        if year == '2015':
            return (geo, value, 0, 0, 0, 0)
        elif year == '2016':
            return (geo, 0, value, 0, 0, 0)
        elif year == '2017':
            return (geo, 0, 0, value, 0, 0)
        elif year == '2018':
            return (geo, 0, 0, 0, value, 0)
        elif year == '2019':
            return (geo, 0, 0, 0, 0, value)
        else:
            return ('', 0, 0, 0, 0, 0)
    except:
        return ('', 0, 0, 0, 0, 0)
    

trend_data = av_data \
    .map(lambda line: parseLine(line)) \
    .map(lambda x: (x[0], (x[1], x[2], x[3], x[4], x[5]))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1], x[2] + y[2], x[3] + y[3], x[4] + y[4]))

#year_data.take(100)

In [3]:
#Anser Question1: temperature trends over the past 5 years.

import numpy as np
a = np.array(trend_data.collect())

for i in a:
    print('-----------------')
    print('geohash: ' + i[0] + '. Five years trend: ' + str(i[1]))
    print('-----------------')

-----------------
geohash: . Five years trend: (0, 0, 0, 0, 0)
-----------------
-----------------
geohash: 9t. Five years trend: (292.3740264019512, 292.0545433190392, 292.7062824710396, 292.40777327929044, 292.17809724301696)
-----------------
-----------------
geohash: b9. Five years trend: (281.94971430927603, 281.8657709841434, 280.8322968986304, 281.53038880701735, 282.33973934571424)
-----------------
-----------------
geohash: 9d. Five years trend: (301.21274428900006, 300.89027061326294, 300.5322136974001, 300.8009366723187, 300.3252629504862)
-----------------
-----------------
geohash: 8g. Five years trend: (297.69041888657705, 296.9503182832287, 296.97685361751866, 297.20653801726144, 297.20918538700556)
-----------------
-----------------
geohash: dz. Five years trend: (283.6980253782013, 283.6951032556055, 282.39626633726533, 282.6079796363637, 282.95916053800755)
-----------------
-----------------
geohash: 9x. Five years trend: (282.14881158584683, 280.75218591312796, 2

In [4]:
#Find out geohash with increasing temperatures
#We regard regions that avrage temperature of 2019 is great than 2015 as "regions with increasing temperatures"

def checkIncreasing(line):
    try:
        #if line[0] < line[1] and line[1] < line[2] and line[2] < line[3] and line[3] < line[4]:
        if line[0] < line[4]:
            return True
        else:
            return False
    except:
        return False
    
for i in a:
    if checkIncreasing(i[1]):
        print(i[0])

b9
f6
9g
b8
f9
fd
dn
9u
fc


In [ ]:
#Now, we try to compute the PCC  

'''
geohashs with increasing temperatures:
b9
f6
9g
b8
f9
fd
dn
9u
fc
'''
import geohash
from datetime import datetime

def parseLine(line):
    variables = line.split("\t")
    try:
        lat = float(variables[1])
        lon = float(variables[2])
        tem = float(variables[10])
        humidity = float(variables[8])
        wind_speed = float(variables[17])
        cloud_cover = float(variables[12])   
        
        ts = int(variables[0][0: 10])
        # if you encounter a "year is out of range" error the timestamp
        # may be in milliseconds, try `ts /= 1000` in that case
        yearMonth = datetime.utcfromtimestamp(ts).strftime('%Y-%m')
        
        gh = geohash.encode(lat, lon)
        return (gh[0: 2] + '\t' + yearMonth, tem, humidity, wind_speed, cloud_cover)
    except:
        return ('', 0, 0, 0, 0)
    
    
def isIncreaseGeo(line):
    try:
        #if line[0] < line[1] and line[1] < line[2] and line[2] < line[3] and line[3] < line[4]:
        if line.startswith('b9') or line.startswith('f6') or line.startswith('9g') or line.startswith('b8') \
        or line.startswith('f9') or line.startswith('fd') or line.startswith('dn') or line.startswith('9u') \
        or line.startswith('fc'):
            return True
        else:
            return False
    except:
        return False
    
#text_file = spark.read.load('hdfs://orion11:21001/3hr_sample/sampled_2015/*', format='csv', sep='\t', inferSchema=True, header=True)
text_fileCC = sc.textFile("hdfs://orion11:21001/3hr_sample/*")

# (GeoHash, wind_energy_factor, cloud_cover)
parsed_dataCC = text_fileCC \
    .map(lambda line: parseLine(line)) \
    .filter(lambda line: isIncreaseGeo(line[0]))

av_data = parsed_dataCC.map(lambda x: (x[0], (x[1], x[2], x[3], x[4]))) \
   .mapValues(lambda x: (x, 1)) \
   .reduceByKey(lambda x, y: (((x[0][0] + y[0][0]), (x[0][1] + y[0][1]), (x[0][2] + y[0][2]), (x[0][3] + y[0][3])), x[1]+y[1])) \
   .mapValues(lambda v: (v[0][0] / v[1], v[0][1] / v[1], v[0][2] / v[1], v[0][3] / v[1]))

#geo-time, average_temperature, average_humidity, average_wind_speed, average_cloud_cover
av_data = av_data.sortBy(lambda x: x[0], True)

import numpy as np
a = np.array(av_data.collect())

#geo - average_temperature in time order
temperature_array = {} 

for i in a:
    if len(i[0]) > 0:
        temperature_array.setdefault(i[0][0: 2], []).append(i[1][0])
        
#geo - average_humidity in time order
humidity_array = {} 

for i in a:
    if len(i[0]) > 0:
        humidity_array.setdefault(i[0][0: 2], []).append(i[1][1])

#geo - average_wind_speed in time order
wind_speed_array = {} 

for i in a:
    if len(i[0]) > 0:
        wind_speed_array.setdefault(i[0][0: 2], []).append(i[1][2])
        
#geo - average_cloud_cover in time order
cloud_cover_array = {} 

for i in a:
    if len(i[0]) > 0:
        cloud_cover_array.setdefault(i[0][0: 2], []).append(i[1][3])


In [5]:
#Answer to question2: PCC of temperature-humidity, temperature-wind_speed, and temperature-could_cover
import numpy as np

for key in temperature_array.keys():
    print('-----------------')
    print('geohash: ' + key)
    print('')
    
    print('humidity: ')
    pccs = np.corrcoef(temperature_array[key], humidity_array[key])
    print(pccs)
    print('')

    print('wind_speed: ')
    pccs = np.corrcoef(temperature_array[key], wind_speed_array[key])
    print(pccs)
    print('')

    print('could_cover: ')
    pccs = np.corrcoef(temperature_array[key], cloud_cover_array[key])
    print(pccs)
    print('')

-----------------
geohash: 9g

humidity: 
[[1.         0.67459864]
 [0.67459864 1.        ]]

wind_speed: 
[[ 1.         -0.16198112]
 [-0.16198112  1.        ]]

could_cover: 
[[ 1.         -0.26594592]
 [-0.26594592  1.        ]]

-----------------
geohash: 9u

humidity: 
[[1.         0.65556102]
 [0.65556102 1.        ]]

wind_speed: 
[[ 1.         -0.56628449]
 [-0.56628449  1.        ]]

could_cover: 
[[ 1.         -0.82594178]
 [-0.82594178  1.        ]]

-----------------
geohash: b8

humidity: 
[[ 1.         -0.83898729]
 [-0.83898729  1.        ]]

wind_speed: 
[[ 1.         -0.27185404]
 [-0.27185404  1.        ]]

could_cover: 
[[1.         0.15790191]
 [0.15790191 1.        ]]

-----------------
geohash: b9

humidity: 
[[ 1.         -0.80385473]
 [-0.80385473  1.        ]]

wind_speed: 
[[ 1.         -0.27180948]
 [-0.27180948  1.        ]]

could_cover: 
[[1.         0.41604289]
 [0.41604289 1.        ]]

-----------------
geohash: dn

humidity: 
[[ 1.         -0.68849882]